# train

In [1]:
import yaml
import os

with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

/home/hiroki/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """


In [2]:
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
os.makedirs(OUTPUT_ROOT, exist_ok=True)

## load library

In [3]:
# python default library
import os
import shutil
import datetime
import sys
import pickle

# general analysis tool-kit
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# pytorch
import torch
from torch import optim, nn
from torch.utils.tensorboard import SummaryWriter

from torchsummary import summary

# etc
import yaml
yaml.warnings({'YAMLLoadWarning': False})
import mlflow
from collections import defaultdict
from scipy.stats import zscore

# original library

import common as com
import pytorch_modeler as modeler
from pytorch_model import CNN6AutoEncoder as Model
from pytorch_utils import filtered_load_model
import models

import librosa
import IPython
import librosa.display

## load config and set logger

In [4]:
with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

log_folder = config['IO_OPTION']['OUTPUT_ROOT']+'/{0}.log'.format(datetime.date.today())
logger = com.setup_logger(log_folder, '00_train.py')

/home/hiroki/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Setting

In [5]:
# Setting seed
modeler.set_seed(42)

In [6]:
############################################################################
# Setting I/O path
############################################################################
# input dirs
INPUT_ROOT = config['IO_OPTION']['INPUT_ROOT']
dev_path = INPUT_ROOT + "/dev_data"
add_dev_path = INPUT_ROOT + "/add_dev_data"
# machine type
MACHINE_TYPE = config['IO_OPTION']['MACHINE_TYPE']
machine_types = os.listdir(dev_path)
# output dirs
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
MODEL_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/models'
TB_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/tb'
PKL_DIR = OUTPUT_ROOT + '/pkl'
#os.makedirs(OUTPUT_ROOT, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(TB_DIR, exist_ok=True)
os.makedirs(PKL_DIR, exist_ok=True)
# copy config
shutil.copy('./config.yaml', OUTPUT_ROOT)

'/media/hiroki/working/research/dcase2020/result/2D/CONV_AE2/config.yaml'

## make path list and train/valid split

In [7]:
############################################################################
# make path set and train/valid split
############################################################################
'''
train_paths[machine_type]['train' or 'valid'] = path
'''
dev_train_paths = {}
add_train_paths = {}
train_paths = {}

In [8]:
for machine_type in machine_types:
    # dev train
    dev_train_paths = ["{}/{}/train/".format(dev_path, machine_type) + file for file in os.listdir("{}/{}/train".format(dev_path, machine_type))]
    dev_train_paths = sorted(dev_train_paths)
    # add_dev train
    add_train_paths = ["{}/{}/train/".format(add_dev_path, machine_type) + file for file in os.listdir("{}/{}/train".format(add_dev_path, machine_type))]
    add_train_paths = sorted(add_train_paths)
    # valid
    dev_valid_paths = ["{}/{}/test/".format(dev_path, machine_type) + file for file in os.listdir("{}/{}/test".format(dev_path, machine_type))]
    dev_valid_paths = sorted(dev_valid_paths)
    
    train_paths[machine_type] = {}
    train_paths[machine_type]['train'] = dev_train_paths + add_train_paths
    train_paths[machine_type]['valid'] = dev_valid_paths

## training

In [10]:
#############################################################################
# run
#############################################################################
def run(machine_type):
    com.tic()
    logger.info('TARGET MACHINE_TYPE: {0}'.format(machine_type))
    logger.info('MAKE DATA_LOADER')
    # dev_train_paths
    dataloaders_dict = modeler.make_dataloader(train_paths, machine_type)
    # define writer for tensorbord
    os.makedirs(TB_DIR+'/'+machine_type, exist_ok=True)         # debug
    tb_log_dir = TB_DIR + '/' + machine_type
    writer = SummaryWriter(log_dir = tb_log_dir)
    logger.info('TRAINING')
    # parameter setting
    net = Model(sample_rate=config['preprocessing']['sample_rate'],
            window_size=config['preprocessing']['window_size'],
            hop_size=config['preprocessing']['hop_size'],
            mel_bins=config['preprocessing']['mel_bins'],
            fmin=config['preprocessing']['fmin'],
            fmax=config['preprocessing']['fmax'])
    #pretrained_dict = torch.load(config['IO_OPTION']['PREMODEL_PATH'])
    #net = filtered_load_model(net, pretrained_dict)
    optimizer = optim.Adam(net.parameters())
    criterion = nn.MSELoss()
    num_epochs = config['fit']['num_epochs']
    history = modeler.train_net(net, dataloaders_dict, criterion, optimizer, num_epochs, writer)
    # output
    model = history['model']
    model_out_path = MODEL_DIR+'/{}_model.pth'.format(machine_type)
    torch.save(model.state_dict(), model_out_path)
    logger.info('\n success:{0} \n'.format(machine_type) + \
                    'model_out_path ==> \n {0}'.format(model_out_path))
    #  close writer for tensorbord
    writer.close()
    #modeler.mlflow_log(history, config, machine_type, model_out_path, tb_log_dir)
    com.toc()
    return history

In [11]:
machine_types

['fan', 'pump', 'slider', 'ToyCar', 'ToyConveyor', 'valve']

In [12]:
for machine_type in machine_types:
    #if (machine_type == machine_types[0]) or (machine_type == machine_types[1]):
    #    continue
    history = run(machine_type)
    with open('{}/{}_history.pkl'.format(PKL_DIR, machine_type), 'wb') as file:
        pickle.dump(history , file)

2020-11-01 11:24:02,408 - 00_train.py - INFO - TARGET MACHINE_TYPE: fan
2020-11-01 11:24:02,409 - 00_train.py - INFO - MAKE DATA_LOADER
2020-11-01 11:24:03,801 - 00_train.py - INFO - TRAINING


use: cuda:0


100%|██████████| 1803/1803 [00:05<00:00, 326.82it/s]
2020-11-01 11:25:40,115 - pytorch_modeler.py - INFO - Epoch 1/100:train_loss:700.432020, valid_AUC:0.520368, valid_pAUC:0.502851
100%|██████████| 1803/1803 [00:05<00:00, 310.93it/s]
2020-11-01 11:26:15,076 - pytorch_modeler.py - INFO - Epoch 2/100:train_loss:199.138204, valid_AUC:0.481877, valid_pAUC:0.507681
100%|██████████| 1803/1803 [00:05<00:00, 309.04it/s]
2020-11-01 11:26:50,292 - pytorch_modeler.py - INFO - Epoch 3/100:train_loss:26.913892, valid_AUC:0.515641, valid_pAUC:0.516328
100%|██████████| 1803/1803 [00:05<00:00, 305.58it/s]
2020-11-01 11:27:26,374 - pytorch_modeler.py - INFO - Epoch 4/100:train_loss:7.383266, valid_AUC:0.506418, valid_pAUC:0.514555
100%|██████████| 1803/1803 [00:05<00:00, 312.58it/s]
2020-11-01 11:28:02,152 - pytorch_modeler.py - INFO - Epoch 5/100:train_loss:7.025315, valid_AUC:0.508807, valid_pAUC:0.512117
100%|██████████| 1803/1803 [00:05<00:00, 313.53it/s]
2020-11-01 11:28:37,792 - pytorch_modeler.

100%|██████████| 1803/1803 [00:10<00:00, 168.90it/s]
2020-11-01 11:49:21,062 - pytorch_modeler.py - INFO - Epoch 37/100:train_loss:5.112770, valid_AUC:0.551974, valid_pAUC:0.505552
100%|██████████| 1803/1803 [00:10<00:00, 166.68it/s]
2020-11-01 11:50:21,559 - pytorch_modeler.py - INFO - Epoch 38/100:train_loss:5.111600, valid_AUC:0.561197, valid_pAUC:0.504605
100%|██████████| 1803/1803 [00:10<00:00, 165.06it/s]
2020-11-01 11:51:22,361 - pytorch_modeler.py - INFO - Epoch 39/100:train_loss:5.112708, valid_AUC:0.548895, valid_pAUC:0.504230
100%|██████████| 1803/1803 [00:10<00:00, 167.01it/s]
2020-11-01 11:52:23,025 - pytorch_modeler.py - INFO - Epoch 40/100:train_loss:5.077705, valid_AUC:0.542177, valid_pAUC:0.499072
100%|██████████| 1803/1803 [00:10<00:00, 165.46it/s]
2020-11-01 11:53:24,171 - pytorch_modeler.py - INFO - Epoch 41/100:train_loss:5.073180, valid_AUC:0.552200, valid_pAUC:0.503789
100%|██████████| 1803/1803 [00:10<00:00, 165.16it/s]
2020-11-01 11:54:24,453 - pytorch_modeler.

100%|██████████| 1803/1803 [00:06<00:00, 287.64it/s]
2020-11-01 12:13:33,212 - pytorch_modeler.py - INFO - Epoch 73/100:train_loss:4.748700, valid_AUC:0.558076, valid_pAUC:0.508244
100%|██████████| 1803/1803 [00:05<00:00, 321.16it/s]
2020-11-01 12:14:08,822 - pytorch_modeler.py - INFO - Epoch 74/100:train_loss:4.751343, valid_AUC:0.567999, valid_pAUC:0.506377
100%|██████████| 1803/1803 [00:05<00:00, 324.67it/s]
2020-11-01 12:14:44,210 - pytorch_modeler.py - INFO - Epoch 75/100:train_loss:4.726666, valid_AUC:0.566162, valid_pAUC:0.505899
100%|██████████| 1803/1803 [00:05<00:00, 308.30it/s]
2020-11-01 12:15:20,023 - pytorch_modeler.py - INFO - Epoch 76/100:train_loss:4.732691, valid_AUC:0.561534, valid_pAUC:0.504333
100%|██████████| 1803/1803 [00:05<00:00, 318.96it/s]
2020-11-01 12:15:55,380 - pytorch_modeler.py - INFO - Epoch 77/100:train_loss:4.731650, valid_AUC:0.561253, valid_pAUC:0.505955
100%|██████████| 1803/1803 [00:05<00:00, 312.01it/s]
2020-11-01 12:16:33,502 - pytorch_modeler.

...

use: cuda:0


100%|██████████| 855/855 [00:06<00:00, 123.07it/s]
2020-11-01 16:27:06,588 - pytorch_modeler.py - INFO - Epoch 1/100:train_loss:776.677389, valid_AUC:0.489758, valid_pAUC:0.503702
100%|██████████| 855/855 [00:02<00:00, 324.38it/s]
2020-11-01 16:27:35,040 - pytorch_modeler.py - INFO - Epoch 2/100:train_loss:275.683319, valid_AUC:0.467929, valid_pAUC:0.490283
100%|██████████| 855/855 [00:02<00:00, 325.29it/s]
2020-11-01 16:28:03,520 - pytorch_modeler.py - INFO - Epoch 3/100:train_loss:51.126964, valid_AUC:0.460302, valid_pAUC:0.497224
100%|██████████| 855/855 [00:02<00:00, 325.55it/s]
2020-11-01 16:28:32,049 - pytorch_modeler.py - INFO - Epoch 4/100:train_loss:14.846250, valid_AUC:0.449912, valid_pAUC:0.496761
100%|██████████| 855/855 [00:02<00:00, 311.83it/s]
2020-11-01 16:29:00,703 - pytorch_modeler.py - INFO - Epoch 5/100:train_loss:11.047036, valid_AUC:0.451407, valid_pAUC:0.495922
100%|██████████| 855/855 [00:02<00:00, 324.28it/s]
2020-11-01 16:29:29,220 - pytorch_modeler.py - INFO 

100%|██████████| 855/855 [00:02<00:00, 322.29it/s]
2020-11-01 16:44:15,557 - pytorch_modeler.py - INFO - Epoch 37/100:train_loss:5.838257, valid_AUC:0.525676, valid_pAUC:0.503470
100%|██████████| 855/855 [00:02<00:00, 323.04it/s]
2020-11-01 16:44:44,086 - pytorch_modeler.py - INFO - Epoch 38/100:train_loss:5.814946, valid_AUC:0.549602, valid_pAUC:0.503065
100%|██████████| 855/855 [00:02<00:00, 321.95it/s]
2020-11-01 16:45:12,729 - pytorch_modeler.py - INFO - Epoch 39/100:train_loss:5.770661, valid_AUC:0.541670, valid_pAUC:0.503528
100%|██████████| 855/855 [00:02<00:00, 322.01it/s]
2020-11-01 16:45:41,325 - pytorch_modeler.py - INFO - Epoch 40/100:train_loss:5.725610, valid_AUC:0.541626, valid_pAUC:0.506217
100%|██████████| 855/855 [00:02<00:00, 321.33it/s]
2020-11-01 16:46:09,891 - pytorch_modeler.py - INFO - Epoch 41/100:train_loss:5.740750, valid_AUC:0.539720, valid_pAUC:0.506998
100%|██████████| 855/855 [00:02<00:00, 321.84it/s]
2020-11-01 16:46:38,524 - pytorch_modeler.py - INFO - 

100%|██████████| 855/855 [00:02<00:00, 319.85it/s]
2020-11-01 17:01:25,206 - pytorch_modeler.py - INFO - Epoch 73/100:train_loss:5.322596, valid_AUC:0.553168, valid_pAUC:0.507721
100%|██████████| 855/855 [00:02<00:00, 323.12it/s]
2020-11-01 17:01:53,717 - pytorch_modeler.py - INFO - Epoch 74/100:train_loss:5.356664, valid_AUC:0.550324, valid_pAUC:0.508762
100%|██████████| 855/855 [00:02<00:00, 323.39it/s]
2020-11-01 17:02:22,245 - pytorch_modeler.py - INFO - Epoch 75/100:train_loss:5.314707, valid_AUC:0.551104, valid_pAUC:0.511625
100%|██████████| 855/855 [00:02<00:00, 324.69it/s]
2020-11-01 17:02:50,794 - pytorch_modeler.py - INFO - Epoch 76/100:train_loss:5.312211, valid_AUC:0.550538, valid_pAUC:0.511625
100%|██████████| 855/855 [00:02<00:00, 321.16it/s]
2020-11-01 17:03:19,278 - pytorch_modeler.py - INFO - Epoch 77/100:train_loss:5.298903, valid_AUC:0.554500, valid_pAUC:0.513505
100%|██████████| 855/855 [00:02<00:00, 323.06it/s]
2020-11-01 17:03:47,858 - pytorch_modeler.py - INFO - 

elapsed time: 2919.694108486 [sec]


history = run('ToyCar')
with open('{}/{}_history.pkl'.format(PKL_DIR, machine_type), 'wb') as file:
    pickle.dump(history , file)

In [12]:
history['reconstruct_img']['label']

/home/hiroki/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0)]